In [2]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
from collections import deque
import time

# --- CONFIG ---
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
I_EAR_THRESH = 0.28     # adjust after calibration
CONSEC_FRAMES = 10        # frame-based alert
SMOOTH_WINDOW = 5
BLINK_DURATION = 2.0      # seconds eyes closed => drowsy

LEFT_EYE = list(range(36, 42))
RIGHT_EYE = list(range(42, 48))

# --- EAR computation ---
def eye_aspect_ratio(eye):
    if len(eye) != 6:
        return 0.0
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C) if C > 0 else 0.0

# Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# dlib shape predictor
predictor = dlib.shape_predictor(PREDICTOR_PATH)

# --- Video capture ---
cap = cv2.VideoCapture(0)
counter, alarm_on = 0, False
ear_history = deque(maxlen=SMOOTH_WINDOW)

# For blink duration tracking
blink_start_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    # Detect faces
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80), flags=cv2.CASCADE_SCALE_IMAGE
    )

    if len(faces) == 0:
        cv2.putText(frame, "FACE UNKNOWN", (50, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
        counter = 0
        alarm_on = False
        blink_start_time = None

    for (x, y, w, h) in faces:
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        shape = predictor(gray, rect)
        coords = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])

        if coords.shape[0] < 68:
            cv2.putText(frame, "DROWSINESS (no eyes)", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
            continue

        # Eyes
        leftEye = coords[LEFT_EYE]
        rightEye = coords[RIGHT_EYE]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        ear_history.append(ear)
        smooth_ear = np.mean(ear_history)

        # Debug EAR
        print(f"EAR: {smooth_ear:.3f}")
        cv2.putText(frame, f"EAR: {smooth_ear:.2f}", (50, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        # Draw
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.polylines(frame, [leftEye], True, (0, 255, 0), 1)
        cv2.polylines(frame, [rightEye], True, (0, 255, 0), 1)

        # --- Drowsiness logic ---
        if smooth_ear < I_EAR_THRESH:
            counter += 1
            if blink_start_time is None:
                blink_start_time = time.time()

            # Rule 1: Frame counter method
            if counter >= CONSEC_FRAMES:
                alarm_on = True

            # Rule 2: Blink duration method
            blink_duration = time.time() - blink_start_time
            if blink_duration >= BLINK_DURATION:
                alarm_on = True
        else:
            counter = 0
            alarm_on = False
            blink_start_time = None

        if alarm_on:
            cv2.putText(frame, "YOU ARE DROWSY!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Drowsiness Detection (Blink Timer)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


EAR: 0.223
EAR: 0.247
EAR: 0.259
EAR: 0.261
EAR: 0.268
EAR: 0.283
EAR: 0.290
EAR: 0.293
EAR: 0.299
EAR: 0.298
EAR: 0.298
EAR: 0.290
EAR: 0.281
EAR: 0.278
EAR: 0.276
EAR: 0.268
EAR: 0.266
EAR: 0.268
EAR: 0.275
EAR: 0.272
EAR: 0.281
EAR: 0.286
EAR: 0.292
EAR: 0.292
EAR: 0.291
EAR: 0.283
EAR: 0.265
EAR: 0.263
EAR: 0.250
EAR: 0.245
EAR: 0.240
EAR: 0.249
EAR: 0.248
EAR: 0.256
EAR: 0.263
EAR: 0.270
EAR: 0.283
EAR: 0.286
EAR: 0.286
EAR: 0.288
EAR: 0.292
EAR: 0.291
EAR: 0.290
EAR: 0.292
EAR: 0.297
EAR: 0.300
EAR: 0.303
EAR: 0.306
EAR: 0.310
EAR: 0.313
EAR: 0.315
EAR: 0.323
EAR: 0.329
EAR: 0.333
EAR: 0.335
EAR: 0.336
EAR: 0.332
EAR: 0.326
EAR: 0.323
EAR: 0.324
EAR: 0.326
EAR: 0.323
EAR: 0.322
EAR: 0.328
EAR: 0.330
EAR: 0.318
EAR: 0.312
EAR: 0.306
EAR: 0.297
EAR: 0.263
EAR: 0.241
EAR: 0.237
EAR: 0.234
EAR: 0.226
EAR: 0.252
EAR: 0.278
EAR: 0.286
EAR: 0.300
EAR: 0.315
EAR: 0.320
EAR: 0.326
EAR: 0.335
EAR: 0.338
EAR: 0.344
EAR: 0.352
EAR: 0.359
EAR: 0.354
EAR: 0.348
EAR: 0.318
EAR: 0.282
EAR: 0.245